In [ ]:
from pfs.lam.detAnalysis import *
from pfs.lam.fileHandling import *
import glob

In [ ]:
#%matplotlib widget

import matplotlib.pyplot as plt
#plt.style.use('dark_background')
from matplotlib import style
style.use('ggplot')

In [ ]:
cluster = False

In [ ]:
if cluster :
    imgPath = '/net/SRVSTK20C/drp/fmadec/analysis/subaru/sm1/'
    dataPath = "/net/SRVSTK20C/drp/fmadec/analysis/subaru/sm1/"
    drpPath = "/net/SRVSTK20C/drp/cluster"
    repo = "repo"
else:
    imgPath = '/media/fmadec/PfsData/PFS/fmadec/analysis/subaru/sm1/'
    dataPath = imgPath
    drpPath = "/home/fmadec/pfs/subaruData/"    
    repo = "sm1-newfoc-20200721"
    repo = "sm1"

In [ ]:
verbose = True

In [ ]:
experimentId = 2605  #248 # 255
rerun = "sm1-focus-sep2020" #"sm1-march2020"
cam = "r1"
roi_size = 24
doBck = True
Sources = ["argon"] #, "Neon"]

In [ ]:
ExpIds = [2605, 2606]  #248 # 255
rerun = "sm1-focus-sep2020" #"sm1-march2020"
cam = "r1"
roi_size = 24
doBck = True

Sources = ["argon", "neon"] #, "Neon"]


In [ ]:
experimentId = 2600 #248 # 255
rerun = "sm1-focus-sep2020" #"sm1-march2020"
cam = "m1"
roi_size = 24
doBck = True

Sources = ["neon"] #, "Neon"]


In [ ]:
#repo = "drp"
#drpPath = "/home/fmadec/pfs/subaruData/"
arm = cam[0]
sm = int(cam[1])
print(arm, sm)

In [ ]:
SM = "SM1"
site = "Subaru"

In [ ]:
ExpIds = [experimentId] #, 250]

In [ ]:
#extra = "Imqual_LAM_peaklist_2020July"
extra = ""

In [ ]:
files = []
for ExpId in ExpIds :
    csvPath = imgPath+"Exp"+str(ExpId)+"/"+rerun+"_roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra
    dataPath = csvPath
#    print(dataPath)

    searchFile = f"{csvPath}Imquality_{cam}_Exp{ExpId}*"
    print(searchFile)
    files.extend(glob.glob(f"{searchFile}"))

if verbose:
    print(*files, sep="\n")

In [ ]:
filelist = files

In [ ]:
comment = extra + f"doBck{doBck}"
savePlotFile = imgPath+f"{SM}_{site}_{cam}_ExpIds_{'-'.join(str(x) for x in ExpIds)}_{'-'.join(str(x) for x in Sources)}"
savePlotFile = savePlotFile+f"_{comment}"
print(savePlotFile)
savePlotScatterFile = imgPath+f"{SM}_{site}_{cam}_ExpIds_{'-'.join(str(x) for x in ExpIds)}_{'-'.join(str(x) for x in Sources)}_scatter"
savePlotScatterFile = savePlotScatterFile+f"_{comment}"
print(savePlotScatterFile)
title = f"{SM} {site} {cam} ExpIds {' '.join(str(x) for x in ExpIds)} {' '.join(str(x) for x in Sources)}"
title = title + f"\n{comment}"
print(title)

In [ ]:
ob = pd.concat([pd.read_csv(filen) for filen in filelist ], ignore_index=True)
#ob = pd.concat(map(pd.read_csv,files)).reset_index().sort_values(by="motor1")

In [ ]:
ob["sep_2ndM"]= ob.apply(lambda x: np.mean([x["sep_x2"],x["sep_y2"]]) , axis=1)

In [ ]:
ob

In [ ]:
ob.element.unique()

In [ ]:
ob.wavelength.unique()

In [ ]:
filelist[0].split('/')[-1]

In [ ]:
def plotImageQuality(dframe, vmin=-1,vmax=-1, par="EE3", hist=None, savePlotFile=None, com=False, doSave=False, title=None ):
    
    # select peak center 
    # default x , y are objx and objy, but if it is the center of Mass it is oid_x and oid_y
#    x = dframe["objx"]
#    y = dframe["objy"]
#    if com :
#        x = dframe["oid_x"]
#        y = dframe["oid_y"]

    ## should now be px, py which are affected during calculation according com or not
    x = dframe["px"]
    y = dframe["py"]    
    
    z = dframe[par]
#    stat = 'ECE5' if par == 'ECE5' else 'EE5'
    val = 0.5 if par == "EE3" else 0.9

    stat = par
    xs = dframe[dframe[stat]>val].px
    ys = dframe[dframe[stat]>val].py
    zs = dframe[dframe[stat]>val][stat]

    statEE = f"{100*len(zs)/len(z):.1f}% of peak have a {par} > {val}"
    # Set up a regular grid of interpolation points
    xi, yi = np.linspace(x.min(), x.max(), 100), np.linspace(y.min(), y.max(), 100)
    xi, yi = np.meshgrid(xi, yi)

    # Interpolate
    rbf = scipy.interpolate.Rbf(x, y, z, function='linear')
    zi = rbf(xi, yi)

    if vmin == -1 :
        vmin = z.min()
    if vmax == -1 :
        vmax = z.max()
    if hist is not None:
        #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
        fig = plt.figure(figsize=(16, 8))
        gs = gridspec.GridSpec(1, 3,
 #                      width_ratios=[3,1],
 #                      height_ratios=[1,1]
                       )
        ax1 = plt.subplot(gs[0,:2])
        ax2 = plt.subplot(gs[0,2])
        im = ax1.imshow(zi, vmin=vmin, vmax=vmax, origin='lower',
                   extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.3)
        ax1.scatter(x, y, c=z, vmin=vmin, vmax=vmax)
        ax1.set_title(par)
        ax1.set_xlim([0,4095])
        ax1.set_ylim([0,4175])
        fig.colorbar(im, ax=ax1, shrink=1)
        dframe[hist].plot.hist(ax=ax2, bins=20)
        ax2.axvline(x=val, color='k')
        ax2.set_title("Histogram")
        if title is not(None):
            fig.suptitle(title+"\n"+statEE)
        
    else : 
        fig = plt.figure(figsize=(8, 8))

        plt.imshow(zi, vmin=vmin, vmax=vmax, origin='lower',
                   extent=[x.min(), x.max(), y.min(), y.max()], alpha=0.3)
        plt.scatter(x, y, c=z, vmin=vmin, vmax=vmax)
        
        plt.colorbar(shrink=0.65)
        plt.scatter(xs, ys, c="r", marker="x")

        plt.xlim(0,4095)
        plt.ylim(0,4175)
        if title is not(None):
            fig.suptitle(title+"\n"+statEE)
          
    if doSave:
        fig.patch.set_alpha(0.5)
        plt.savefig(savePlotFile+f"_{par}.png")
        plt.show()

In [ ]:
def plotOnePeak(image, cx,cy, roi_size=30, doBck=False, nRows=5, vmin=None, vmax=None):
    indx = cy
    indy = cx

    if type(image) is str:
        hdulist = fits.open(image, "readonly")
        image = hdulist[1].data

    data = np.copy(image)
    outer_data = data[int(indx-roi_size/2):int(indx+roi_size/2), int(indy-roi_size/2):int(indy+roi_size/2)]
    
    if doBck:
        lowRows = image[int(indx-nRows-roi_size/2):int(indx-roi_size/2), int(indy-roi_size/2):int(indy+roi_size/2)]
        upRows = image[int(indx+roi_size/2):int(indx+nRows+roi_size/2), int(indy-roi_size/2):int(indy+roi_size/2)]
        allRows = np.zeros((2*nRows, roi_size))
        allRows[:nRows,:] = lowRows
        allRows[-nRows:,:] = upRows
        for i in range(roi_size):
            outer_data[:,i] -= np.median(allRows[:,i])

    vmin = vmin if vmin is not None else np.amin(outer_data)
    vmax = vmax if vmax is not None else np.amax(outer_data)
    
    ax = plt.subplot(111)
    im = ax.imshow(outer_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
#ob.fwhm.plot()
ob.brightness.plot()
print(ob.brightness.min())
print(ob.brightness.max())

In [ ]:
plt.figure()
#ob.fwhm.plot(ylim=[-5,100])
ob.fwhm.plot()

In [ ]:
plt.figure()
ax= ob.fwhm.hist(bins=20, figsize=(10,10), )
#ax.axvline(x=0.9, color='k')
#ax.set_xlim(0.5,1)

In [ ]:
ax= ob.brightness.hist(bins=50, figsize=(10,10), )
#ax.axvline(x=0.9, color='k')
#ax.set_xlim(0.5,1)

In [ ]:
ob.EE5.hist(bins=50, figsize=(10,10), )

# do some filtering if needed/wanted

In [ ]:
df = ob

In [ ]:
len(ob[(ob.sep_flag >1) ])

In [ ]:
df = ob[(ob.sep_flag <1) ]

In [ ]:
df = ob[(ob.sep_flag <1) & (ob.fwhm_x >1.6) & (ob.fwhm_y >1.6) & (ob.fwhm < 10) ]

In [ ]:
df = ob[(ob.sep_flag <1) & (ob.fwhm_x >2.6) & (ob.fwhm < 10) ]

In [ ]:
df = ob[(ob.sep_flag <1) & (ob.fwhm > 2) ]

In [ ]:
df = ob[(ob.sep_flag <1) & (ob.fwhm_x >1.6) & (ob.fwhm_y >1.6) & (ob.fwhm < 10) & (ob.brightness > 5000)]

In [ ]:
df = df[df.EE5>0.3]  # the only point below that in Exp248 is because of CR

In [ ]:
ax= df.brightness.hist(bins=50, figsize=(10,10), )

In [ ]:
df.brightness.plot()
print(df.brightness.min())

In [ ]:
df.EE5.hist(bins=50)

In [ ]:
plt.figure()
df.fwhm.plot()

In [ ]:
len(df)

# find the best EE5 value from the dither sequence

In [ ]:
best = []
for key,group in df.groupby(["wavelength","fiber"]):
#    best.append(group.where(group.EE5 == group.EE5.max()))
    best.append(group[group.EE5 == group.EE5.max()])
#    print(group.px.std())
#    print(group.py.std())

    #best.append(key)
best = pd.concat(best)

In [ ]:
len(best)

In [ ]:
doSavePlot = True

In [ ]:
plotImageQualityScatter(best, par="EE5", hist="EE5", savePlotFile=savePlotScatterFile+'_best', title=title+" best", doSave=doSavePlot,\
                       vmin=-1, vmax=-1)

In [ ]:
plotImageQualityScatter(best, par="EE3", hist="EE3", savePlotFile=savePlotScatterFile+'_best', title=title+" best", doSave=doSavePlot,\
                       vmin=-1, vmax=-1)

In [ ]:
plotImageQualityScatter(best, par="brightness", hist="EE5", savePlotFile=savePlotScatterFile, title=title, doSave=doSavePlot,\
                       vmin=-1, vmax=-1)

In [ ]:
plotImageQualityScatter(df, par="EE5", hist="EE5", savePlotFile=savePlotScatterFile, title=title, doSave=doSavePlot)

In [ ]:
plotImageQualityScatter(df, par="EE3", hist="EE3", savePlotFile=savePlotScatterFile, title=title,
                        vmin=-1, vmax=-1,
                        doSave=True)

In [ ]:
plotImageQualityScatter(df, par="brightness", hist="EE5", savePlotFile=savePlotScatterFile, title=title,
                        vmin=-1, vmax=-1,
                        doSave=False)

In [ ]:
plotImageQualityScatter(df, par="sep_2ndM", hist="sep_2ndM",\
                        savePlotFile=savePlotScatterFile+"_2ndM", title=title,\
                        vmin=-1, vmax = -1,\
                        doSave=True)

In [ ]:
plotImageQualityScatter(ob, par="EE5", hist="EE5", savePlotFile=savePlotScatterFile, title=title, doSave=False)

In [ ]:
ob["sep_meanEC5"]= ob.apply(lambda x: np.mean([x["sep_ECE5"],x["sep_ECR5"]]) , axis=1)

In [ ]:
plotImageQualityScatter(ob, par="sep_meanEC5", hist="EE5", savePlotFile=savePlotScatterFile, title=title, doSave=False)

In [ ]:
plotImageQualityScatter(best, par="EE3", hist="EE3", savePlotFile=savePlotScatterFile+'_best', title=title+' best', doSave=doSavePlot,\
                       vmin=-1, vmax=-1)

In [ ]:
plotImageQualityScatter(df, par="EE3", hist="EE3", savePlotFile=savePlotScatterFile, title=title, doSave=doSavePlot,\
                       vmin=-1, vmax=-1)

In [ ]:
df.sep_xy

In [ ]:
getStatIM(df,par='EE5', thresold=0.82)

In [ ]:
df.head()

In [ ]:
ob.fwhm.describe()

In [ ]:
pdf = df[(df.EE5<0.83)]
print(len(pdf))

In [ ]:
pdf.set_index(["wavelength", "fiber"])

In [ ]:
pdf.fwhm_y

In [ ]:
pdf.sep_flag

In [ ]:
ind = 0

In [ ]:
visitId = int(pdf.visit.iloc[ind])
print(visitId)
print(pdf.wavelength.iloc[ind])
print(pdf.fiber.iloc[ind])
#visitId = 3449 

In [ ]:
import lsst.daf.persistence as dafPersist

In [ ]:
print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")

In [ ]:
butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
calExp = butler.get("calexp", visit=visitId, arm=arm)

In [ ]:
cx = pdf.px.iloc[ind]
cy = pdf.py.iloc[ind]
print(cx,cy)
#imagefile = imgPath+ pdf.file.iloc[ind]
#print(imagefile)

In [ ]:
def plotOnePeak(image, cx,cy, roi_size=30, doBck=False, nRows=5, vmin=None, vmax=None, verbose=False):
    indx = cy
    indy = cx

    if type(image) is str:
        hdulist = fits.open(image, "readonly")
        image = hdulist[1].data
    

    data = np.copy(image)
    outer_data, inner_data = getRois(data, cx, cy, inner_size=5, outer_size=roi_size, doBck=doBck, nRows=nRows)    
    m, s = np.mean(outer_data), np.std(outer_data)
    if verbose:
        print(f"mean: {m}")
        print(f"std: {s}")
    vmin = vmin if vmin is not None else m-s
    vmax = vmax if vmax is not None else m+s
    
    fig, (ax, ax2) = plt.subplots(ncols=2, constrained_layout=True)
    #ax = plt.subplot(111)
    im = ax.imshow(outer_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    
    m = np.log10(np.where(outer_data>0, outer_data, -1))

    #ax2 = plt.subplot(121)
    im = ax2.imshow(m,interpolation="none", origin="lower",cmap="gray")
    
    plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:

plotOnePeak(calExp.image.array,cx, cy, roi_size=24, vmin=0, vmax= 10, doBck=False, verbose=True)

In [ ]:
roi_size = 24
com = True  # Center Of Mass
doBck = doBck # 
head = 0
tail = 0
criteria = 'EE5'
verbose = False
doPrint=True

In [ ]:
getPeakData(calExp.image.array, cx,cy, roi_size=roi_size, doBck=doBck)

In [ ]:
getEE(calExp.image.array, cx,cy, roi_size=roi_size, doBck=doBck, nRows=5, ee_size=5)

In [ ]:
a = getPeakDataSep(calExp.image.array, cx,cy,roi_size=roi_size, EE=[3,5], mask_size=100, threshold=200, doBck=True)

In [ ]:
a.y2

In [ ]:
a = getPeakDataSep(calExp.image.array, cx,cy,roi_size=roi_size, EE=[3,5], mask_size=100, threshold=200, doBck=False)

In [ ]:
a.y2